implementing the metropolis alg on the same example that they use in the 1953 paper

making some assumptions:
* particles can't leave a bounding box (let the bounding box be unit length)
* particles start in a random arrangement
* particles don't have a length or anything like that
* we will deal in 2d

In [56]:
%%capture
! pip install numpy
! pip install matplotlib

In [57]:
import numpy as np
import matplotlib.pyplot as plt

In [91]:
boltz: float = 1.38*(10**-23)
temp: float = 298
max_move: float = 0.05

# 244 was the default used for the unit square in the paper
def generate_particle_set(n:int = 244) -> list[float]:
    return np.random.rand(n,2)


def in_bounds(position: list[float]) -> list[float]: # tbh this is a problem the code gets hung up on this bc we need random chance
    for i,coord in enumerate(position):
        if coord>1:
            position[i] = 1
        if coord<0:
            position[i] = 0 
    return position


def rand_move(position: tuple[float]) -> list[float]:
    new_pos: list = []
    for coord in position:
        new_pos.append(coord+max_move*((np.random.rand(1)[0]*2)-1))
    return in_bounds(new_pos)

# i don't think that this energy equation is right
def calc_e(positions: list[int]) -> float:
    # Constants
    epsilon: float = 1.0  # Energy scale, units in eV (electron volts)
    sigma: float = 1.0    # Distance scale, units in angstroms
    
    total_energy: float = 0.0
    
    # Loop over all pairs of particles
    num_particles = len(positions)
    for i in range(num_particles):
        for j in range(i + 1, num_particles):
            # Calculate distance between particles i and j
            r = np.linalg.norm(np.array(positions[i]) - np.array(positions[j]))
            
            # Calculate Lennard-Jones potential
            v_ij = 4 * epsilon * ((sigma / r) ** 12 - (sigma / r) ** 6)
            
            # Add potential to total energy
            total_energy += v_ij
    
    return total_energy


def accept_move(new_positions: list[float], old_positions: list[float], index: int) -> list[float]:
    old_energy = calc_e(old_positions)
    new_energy = calc_e(new_positions)
    delta_e = new_energy - old_energy
    if delta_e>0:
        return True
    else:
        boltzmann_factor = np.exp(- delta_e / (boltz * temp))
        if np.random.rand() < boltzmann_factor:
            return True
        else:
            return False


def step_index(positions: list[float], i: int) -> list[float]:
    new_positions = positions
    new_positions[i] = rand_move(positions[i])
    return new_positions if accept_move(new_positions, positions, i) else positions


def metro_mcmc_step(old_positions: list[float]) -> list[float]:
    new_positions = old_positions
    for i in range(len(old_positions)):
        new_positions = step_index(new_positions, i)
    return new_positions


def plot_particles(positions: list[float]) -> None:
    print(positions[:,0])
    plt.scatter(positions[:,0],positions[:,1], color='blue')
    plt.xlabel('X')
    plt.ylabel('Y')
    plt.title('Particle Coordinates')
    plt.grid(True)
    plt.box(True)
    plt.xlim(0,1)
    plt.ylim(0,1)
    plt.show()
    return None


particles = generate_particle_set()

for i in range (1000):
    particles = metro_mcmc_step(particles)

plot_particles(particles)

[0.01497059 0.40163363]
1.0
[0.01451243 0.96507115]
1.0
[0.69607783 0.46708473]
1.0
[0.91187815 0.69305032]
1.0
[0.04222243 0.2220631 ]
1.0
[0.82667233 0.54257914]
1.0
[0.34394291 0.30508726]
1.0
[0.34198909 0.70150076]
1.0
[0.26403114 0.49673926]
1.0
[0.83252309 0.88420081]
1.0
[0.97896225 0.48863749]
1.0
[0.20063549 0.20184546]
1.0
[0.75463875 0.92441898]
1.0
[0.04325223 0.32140765]
1.0
[0.96297477 0.19178911]
1.0
[0.78196008 0.94258083]
1.0
[0.50465569 0.72642995]
1.0
[0.60492581 0.47356279]
1.0
[0.12364555 0.22302735]
1.0
[0.82850467 0.66537649]
1.0
[0.78403446 0.4135228 ]
1.0
[0.88793501 0.50472412]
1.0
[0.65679551 0.52361483]
1.0
[0.43224562 0.60522059]
1.0
[0.13647189 0.86883614]
1.0
[0.37571959 0.13370093]
1.0
[0.55346396 0.60890414]
1.0
[0.88390397 0.67919268]
1.0
[0.15680293 0.74125442]
1.0
[0.52182814 0.27066704]
1.0
[0.47116487 0.51121382]
1.0
[0.32636004 0.3918772 ]
1.0
[0.51360277 0.25202031]
1.0
[0.91875829 0.99308217]
1.0
[0.48215375 0.45348741]
1.0
[0.6454358  0.327125

KeyboardInterrupt: 

In [89]:
# in the paper they use 224 for a unit square so why not do the same
particles = generate_particle_set(4)
particles

array([[0.09061251, 0.41171554],
       [0.15063233, 0.68828479],
       [0.30865199, 0.58784158],
       [0.88303064, 0.55170803]])

In [25]:
new_particles = metro_mcmc_step(particles)
print(new_particles)

[nan nan]
[array([nan]), array([nan])]


ValueError: setting an array element with a sequence. The requested array would exceed the maximum number of dimension of 1.